In [1]:
import numpy as np
import pandas as pd
import pandasql
import seaborn as sns
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import feature_selection
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import ShuffleSplit, train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import f1_score
import sklearn.learning_curve as curves
import matplotlib.pyplot as plt
from time import time
import datetime

%matplotlib inline

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

/home/amywu/anaconda3/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/amywu/anaconda3/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/amywu/anaconda3/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  

In [2]:
df = pd.read_csv('employee_retention_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24702 entries, 0 to 24701
Data columns (total 7 columns):
employee_id    24702 non-null float64
company_id     24702 non-null int64
dept           24702 non-null object
seniority      24702 non-null int64
salary         24702 non-null float64
join_date      24702 non-null object
quit_date      13510 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 1.3+ MB


In [3]:
df.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaN
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [4]:
df.describe()

,employee_id,company_id,seniority,salary
count,24702.000000,24702.000000,24702.000000,24702.000000
mean,501604.403530,3.426969,14.127803,138183.345478
std,288909.026101,2.700011,8.089520,76058.184573
min,36.000000,1.000000,1.000000,17000.000000
25%,250133.750000,1.000000,7.000000,79000.000000
50%,500793.000000,2.000000,14.000000,123000.000000
75%,753137.250000,5.000000,21.000000,187000.000000
max,999969.000000,12.000000,99.000000,408000.000000


In [5]:
df.describe(include=[np.object])

,dept,join_date,quit_date
count,24702,24702,13510
unique,6,995,664
top,customer_service,2012-01-03,2015-05-08
freq,9180,105,111


In [6]:
print df.groupby('dept')['salary'].mean().sort_values()
print df.groupby('seniority').count()

dept
customer_service     82245.424837
marketing           135598.042311
sales               135912.358134
design              137460.869565
engineer            205544.548016
data_science        206885.893417
Name: salary, dtype: float64
           employee_id  company_id  dept  salary  join_date  quit_date
seniority                                                             
1                  861         861   861     861        861        430
2                  942         942   942     942        942        500
3                  881         881   881     881        881        447
4                  895         895   895     895        895        422
5                  936         936   936     936        936        533
6                  950         950   950     950        950        571
7                  928         928   928     928        928        511
8                 1008        1008  1008    1008       1008        586
9                  944         944   944     944    

In [7]:
df = df[df['seniority'] <= 30]
df.groupby('seniority').count()

,employee_id,company_id,dept,salary,join_date,quit_date
seniority,,,,,,
1,861,861,861,861,861,430
2,942,942,942,942,942,500
3,881,881,881,881,881,447
4,895,895,895,895,895,422
5,936,936,936,936,936,533
6,950,950,950,950,950,571
7,928,928,928,928,928,511
8,1008,1008,1008,1008,1008,586
9,944,944,944,944,944,522


In [8]:
companies = pd.unique(df['company_id'])
company_id = pd.DataFrame(companies, columns = ['company'])
company_id['key'] = 0

date = datetime.date(2011, 1, 24)
end_date = datetime.date(2015, 12, 13)
day_step = datetime.timedelta(days = 1)
days = []

while date <= end_date:
    days.append(date)
    date += day_step

print 'total days from 2011-01-24 to 2015-12-13:\t', len(days)

date = pd.DataFrame(days)
date['key'] = 0   

df_hc_columns = ['date','employee_headcount','company_id']
df_hc = pd.merge(date, company_id, on = 'key')
df_hc.columns = df_hc_columns
df_hc


total days from 2011-01-24 to 2015-12-13:	1785


,date,employee_headcount,company_id
0,2011-01-24,0,7
1,2011-01-24,0,4
2,2011-01-24,0,2
3,2011-01-24,0,9
4,2011-01-24,0,1
5,2011-01-24,0,6
6,2011-01-24,0,10
7,2011-01-24,0,5
8,2011-01-24,0,3
9,2011-01-24,0,8


In [9]:
for key, value in df.groupby(['join_date', 'company_id']):
    print key, len(value['join_date'])
    print key[0], key[1]


('2011-01-24', 1) 25
2011-01-24 1
('2011-01-24', 2) 17
2011-01-24 2
('2011-01-24', 3) 9
2011-01-24 3
('2011-01-24', 4) 12
2011-01-24 4
('2011-01-24', 5) 5
2011-01-24 5
('2011-01-24', 6) 3
2011-01-24 6
('2011-01-24', 7) 1
2011-01-24 7
('2011-01-24', 8) 6
2011-01-24 8
('2011-01-24', 9) 3
2011-01-24 9
('2011-01-25', 1) 2
2011-01-25 1
('2011-01-25', 2) 2
2011-01-25 2
('2011-01-25', 3) 1
2011-01-25 3
('2011-01-26', 1) 2
2011-01-26 1
('2011-01-26', 3) 2
2011-01-26 3
('2011-01-26', 4) 1
2011-01-26 4
('2011-01-26', 5) 1
2011-01-26 5
('2011-01-31', 1) 30
2011-01-31 1
('2011-01-31', 2) 12
2011-01-31 2
('2011-01-31', 3) 7
2011-01-31 3
('2011-01-31', 4) 5
2011-01-31 4
('2011-01-31', 5) 7
2011-01-31 5
('2011-01-31', 6) 7
2011-01-31 6
('2011-01-31', 7) 6
2011-01-31 7
('2011-01-31', 8) 5
2011-01-31 8
('2011-01-31', 9) 3
2011-01-31 9
('2011-01-31', 10) 1
2011-01-31 10
('2011-02-01', 1) 7
2011-02-01 1
('2011-02-01', 2) 2
2011-02-01 2
('2011-02-01', 3) 1
2011-02-01 3
('2011-02-01', 4) 1
2011-02-01 4
('2

In [10]:
df_join = df.groupby(['join_date', 'company_id']).size().to_frame(name = 'join_count').reset_index()
df_join

,join_date,company_id,join_count
0,2011-01-24,1,25
1,2011-01-24,2,17
2,2011-01-24,3,9
3,2011-01-24,4,12
4,2011-01-24,5,5
5,2011-01-24,6,3
6,2011-01-24,7,1
7,2011-01-24,8,6
8,2011-01-24,9,3
9,2011-01-25,1,2


In [11]:
df_quit = df.groupby(['quit_date', 'company_id']).size().to_frame(name = 'quit_count').reset_index()
df_quit

,quit_date,company_id,quit_count
0,2011-10-13,6,1
1,2011-10-14,10,1
2,2011-10-21,1,1
3,2011-10-28,4,1
4,2011-11-11,1,1
5,2011-11-22,1,1
6,2011-11-25,1,1
7,2011-11-25,2,1
8,2011-12-02,6,1
9,2011-12-02,8,1
